In [ ]:
import pandas as pd

In [ ]:
#orders_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/orders.csv'
#products_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/products.csv'
#customers_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/customers.csv'
#regions_url = 'https://raw.githubusercontent.com/JannesPeeters/DEAD/main/data/regions.csv'

#orders = pd.read_csv(orders_url)
#products = pd.read_csv(products_url)
#customers = pd.read_csv(customers_url)
#regions = pd.read_csv(regions_url)

# directly read csv seems to be faster
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
customers = pd.read_csv('data/customers.csv')
regions = pd.read_csv('data/regions.csv')

In [ ]:
#products.Type.unique()

In [ ]:
#customers['Key Account'].unique()

In [ ]:
# Splitting Orders Table by Products
orders[['Products_split']] = orders[['Products']].apply(lambda x: x.str.split(';'))
orders[['productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split']] = orders[['productsIDs', 'Quantities', 'ProductPricesInCP']].apply(lambda x: x.str.split(','))

orders = orders.explode(['Products_split','productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split'])
orders.drop(['Products','productsIDs', 'Quantities', 'ProductPricesInCP'],axis=1,inplace=True)

orders[['productsIDs_split', 'Quantities_splits']] = orders[['productsIDs_split', 'Quantities_splits']] .astype(int)
orders['ProductPricesInCP_split'] = orders['ProductPricesInCP_split'] .astype(float)
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
orders['DeliveryDate'] = pd.to_datetime(orders['DeliveryDate'])

In [ ]:
# Join all other tables

combine = orders.merge(products, left_on ='productsIDs_split', right_on = 'product_code', how = 'left')\
                .merge(customers.drop(['Territory'],axis=1), left_on ='CustomerID', right_on = 'Account Code', how = 'left')\
                .merge(regions, on = 'Territory', how = 'left')

In [ ]:
combine.shape

In [ ]:
pd.set_option('display.max_columns', None)
combine.head()

In [ ]:
combine.to_csv('orders_merge.csv',index=False)
combine.to_json('orders_merge.json', orient='records',index=False)